In [1]:
# ignore this cell, this is just a helper cell to provide the magic %highlight_file
%run ../../highlighter.py

## Inventory

The Inventory is arguably the most important piece of nornir. Let's see how it works. To begin with the [inventory](../../ref/api/inventory.rst#nornir.core.inventory.Inventory) is comprised of [hosts](../../ref/api/inventory.rst#nornir.core.inventory.Host) and [groups](../../ref/api/inventory.rst#nornir.core.inventory.Group).

In this tutorial we are using the [SimpleInventory](../../plugins/inventory/simple.rst#nornir.plugins.inventory.simple.SimpleInventory) plugin. This inventory plugin stores all the relevant data in two files. Let's start by checking them:

In [2]:
# hosts file
%highlight_file inventory/hosts.yaml

The hosts file is basically a map where the outermost key is the hostname and then any arbitrary `<key, value>` pair you want inside. Usually `nornir_*` keys have special meaning, you can investigate the [hosts](../../ref/api/inventory.rst#nornir.core.inventory.Host) class for details on those. In addition, the `groups` key is a list of groups you can inherite data from. We will inspect soon how the inheritance model works.

Now, let's look at the groups file:

In [3]:
# groups file
%highlight_file inventory/groups.yaml

Pretty similar to the hosts file.

### Accessing the inventory

You can access the [inventory](../../ref/api/inventory.rst#nornir.core.inventory.Inventory) with the `inventory` attribute:

In [4]:
from nornir.core import InitNornir
nr = InitNornir(config_file="config.yaml")

nr.inventory

The inventory has two dict-like attributes `hosts` and `groups` that you can use to access the hosts and groups respectively:

In [5]:
nr.inventory.hosts

{'host1.cmh': Host: host1.cmh,
 'host2.cmh': Host: host2.cmh,
 'spine00.cmh': Host: spine00.cmh,
 'spine01.cmh': Host: spine01.cmh,
 'leaf00.cmh': Host: leaf00.cmh,
 'leaf01.cmh': Host: leaf01.cmh,
 'host1.bma': Host: host1.bma,
 'host2.bma': Host: host2.bma,
 'spine00.bma': Host: spine00.bma,
 'spine01.bma': Host: spine01.bma,
 'leaf00.bma': Host: leaf00.bma,
 'leaf01.bma': Host: leaf01.bma}

In [6]:
nr.inventory.groups

{'global': Group: global,
 'eu': Group: eu,
 'bma': Group: bma,
 'cmh': Group: cmh}

In [7]:
nr.inventory.hosts["leaf01.bma"]

Host: leaf01.bma

Hosts and groups are also dict-like objects:

In [8]:
host = nr.inventory.hosts["leaf01.bma"]
host.keys()

dict_keys(['name', 'groups', 'nornir_host', 'nornir_username', 'nornir_password', 'nornir_network_api_port', 'site', 'role', 'nornir_nos', 'type', 'asn', 'domain'])

In [9]:
host["site"]

'bma'

### Inheritance model

Let's see how the inheritance models works by example. Let's start by looking again at the groups file:

In [10]:
# groups file
%highlight_file inventory/groups.yaml

The host `leaf01.bma` belongs to the group `bma` which in turn belongs to the groups `eu` and `global`. The host `spine00.cmh` belongs to the group `cmh` which doesn't belong to any other group.

Data resolution works by iterating recursively over all the parent groups and trying to see if that parent group (or any of it's parents) contains the data. For instance:

In [11]:
leaf01_bma = nr.inventory.hosts["leaf01.bma"]
leaf01_bma["domain"]  # comes from the group `global`

'global.local'

In [12]:
leaf01_bma["asn"]  # comes from group `eu`

65100

The group `defaults` is special. This group contains data that will be returned if neither the host nor the parents have a specific value for it.

In [13]:
leaf01_cmh = nr.inventory.hosts["leaf01.cmh"]
leaf01_cmh["domain"]  # comes from defaults

'acme.local'

If nornir can't resolve the data you should get a KeyError as usual:

In [14]:
try:
    leaf01_cmh["non_existent"]
except KeyError as e:
    print(f"Couldn't find key: {e}")

Couldn't find key: 'non_existent'


You can also try to access data without recursive resolution by using the `data` attribute. For example, if we try to access `leaf01_cmh.data["domain"]` we should get an error as the host itself doesn't have that data:

In [15]:
try:
    leaf01_cmh.data["domain"]
except KeyError as e:
    print(f"Couldn't find key: {e}")

Couldn't find key: 'domain'


### Filtering the inventory

So far we have seen that `nr.inventory.hosts` and `nr.inventory.groups` are dict-like objects that we can use to iterate over all the hosts and groups or to access any particular one directly. Now we are going to see how we can do some fancy filtering that will enable us to operate on groups of hosts based on their properties.

The simpler way of filtering hosts is by `<key, value>` pairs. For instance:

In [16]:
nr.filter(site="cmh").inventory.hosts.keys()

dict_keys(['host1.cmh', 'host2.cmh', 'spine00.cmh', 'spine01.cmh', 'leaf00.cmh', 'leaf01.cmh'])

You can also filter using multiple `<key, value>` pairs:

In [17]:
nr.filter(site="cmh", role="spine").inventory.hosts.keys()

dict_keys(['spine00.cmh', 'spine01.cmh'])

Filter is cumulative:

In [18]:
nr.filter(site="cmh").filter(role="spine").inventory.hosts.keys()

dict_keys(['spine00.cmh', 'spine01.cmh'])

Or:

In [19]:
cmh = nr.filter(site="cmh")
cmh.filter(role="spine").inventory.hosts.keys()

dict_keys(['spine00.cmh', 'spine01.cmh'])

In [20]:
cmh.filter(role="leaf").inventory.hosts.keys()

dict_keys(['leaf00.cmh', 'leaf01.cmh'])

You can also grab the children of a group:

In [21]:
nr.inventory.groups["eu"].children()

{'host1.bma': Host: host1.bma,
 'host2.bma': Host: host2.bma,
 'spine00.bma': Host: spine00.bma,
 'spine01.bma': Host: spine01.bma,
 'leaf00.bma': Host: leaf00.bma,
 'leaf01.bma': Host: leaf01.bma}

#### Advanced filtering

Sometimes you need more fancy filtering. For those cases you have two options:

1. Use a filter function.
2. Use a filter object.

##### Filter functions

The ``filter_func`` parameter let's you run your own code to filter the hosts. The function signature is as simple as ``my_func(host)`` where host is an object of type [Host](../../ref/api/inventory.rst#nornir.core.inventory.Host) and it has to return either ``True`` or ``False`` to indicate if you want to host or not.

In [22]:
def has_long_name(host):
    return len(host.name) == 11

nr.filter(filter_func=has_long_name).inventory.hosts.keys()

dict_keys(['spine00.cmh', 'spine01.cmh', 'spine00.bma', 'spine01.bma'])

In [23]:
# Or a lambda function
nr.filter(filter_func=lambda h: len(h.name) == 9).inventory.hosts.keys()

dict_keys(['host1.cmh', 'host2.cmh', 'host1.bma', 'host2.bma'])

##### Filter Object

You can also use a filter object to create incrementally a complext query object. Let's see how it works by example:

In [24]:
# first you need to import the F object
from nornir.core.filter import F

In [25]:
# hosts in group cmh
cmh = nr.filter(F(groups__contains="cmh"))
print(cmh.inventory.hosts.keys())

dict_keys(['host1.cmh', 'host2.cmh', 'spine00.cmh', 'spine01.cmh', 'leaf00.cmh', 'leaf01.cmh'])


In [26]:
# devices running either linux or eos
linux_or_eos = nr.filter(F(nornir_nos="linux") | F(nornir_nos="eos"))
print(linux_or_eos.inventory.hosts.keys())

dict_keys(['host1.cmh', 'host2.cmh', 'spine00.cmh', 'leaf00.cmh', 'host1.bma', 'host2.bma', 'spine00.bma', 'leaf00.bma'])


In [27]:
# spines in cmh
cmh_and_spine = nr.filter(F(groups__contains="cmh") & F(role="spine"))
print(cmh_and_spine.inventory.hosts.keys())

dict_keys(['spine00.cmh', 'spine01.cmh'])


In [28]:
# cmh devices that are not spines
cmh_and_not_spine = nr.filter(F(groups__contains="cmh") & ~F(role="spine"))
print(cmh_and_not_spine.inventory.hosts.keys())

dict_keys(['host1.cmh', 'host2.cmh', 'leaf00.cmh', 'leaf01.cmh'])
